In [63]:
import pandas as pd
import os
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
import numpy as np
from numpy.linalg import norm
from ast import literal_eval
import gensim.downloader as api

In [179]:
data = pd.read_pickle('/Users/aditya/Documents/GitHub/tegger-challenge/mean_embeddings.pkl')
data.head()

,all,theme,title,contents,argentina,australia,brazil,canada,china,france,...,republic of korea,mexico,russia,saudi arabia,south africa,turkey,united kingdom,united states,european union,mean_emb
0,musician tackle *USA red tape musician group t...,entertainment,Musicians to tackle US red tape,Musicians' groups are to tackle US visa regula...,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,1,"[0.010030553, 0.010555822, 0.0056487704, -0.00..."
1,u2 desire number one u2 three prestigious gram...,entertainment,U2's desire to be number one,"U2, who have won three prestigious Grammy Awar...",0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,"[0.009995017, 0.012596794, 0.0047847545, -0.00..."
2,rocker doherty onstage fight rock singer pete ...,entertainment,Rocker Doherty in on-stage fight,Rock singer Pete Doherty has been involved in ...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0.0095635895, 0.012760582, 0.004244135, -0.00..."
3,snicket top *USA box office chart film adaptat...,entertainment,Snicket tops US box office chart,The film adaptation of Lemony Snicket novels h...,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,"[0.009604343, 0.014471119, 0.0036928602, -0.00..."
4,ocean twelve raid box office ocean twelve crim...,entertainment,Ocean's Twelve raids box office,"Ocean's Twelve, the crime caper sequel starrin...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,"[0.0082686655, 0.012306145, 0.003344359, -0.00..."


In [161]:
g20 = "Argentina, Australia, Brazil, Canada, China, France, Germany, India, Indonesia, Italy, Japan, Korea, Mexico, Russia, Arabia, Africa, Turkey, UK, america, eu".lower()
g20 = g20.split(", ")

In [162]:
# model = Word2Vec.load("w2v_saved.model")

In [163]:
#this is to get average embeds for relevant articles

# g20_mean_emb = {}
# for each in g20:
#     subset = data[data[each]==1]
#     g20_mean_emb[each] = np.mean(subset['mean_emb'], axis = 0)

# this is just embeds for the important words

g20_mean_emb = {}
for each in g20:
    try:
        g20_mean_emb[each] = model['*'+each.upper()]
    except:
        g20_mean_emb[each] = model[each]

In [164]:
g20_mean_emb

{'argentina': array([ 0.01205488,  0.01540329,  0.00587998, ..., -0.03364547,
        -0.02346341,  0.01895906], dtype=float32),
 'australia': array([ 0.0120225 ,  0.01579212,  0.00597072, ..., -0.03384686,
        -0.0229663 ,  0.01896352], dtype=float32),
 'brazil': array([ 0.00753566,  0.00865833,  0.00401651, ..., -0.02044253,
        -0.01490953,  0.01099955], dtype=float32),
 'canada': array([ 0.00579164,  0.00740887,  0.00269136, ..., -0.01600016,
        -0.01082051,  0.00921129], dtype=float32),
 'china': array([ 0.01522709,  0.01920916,  0.00810206, ..., -0.04266999,
        -0.02943521,  0.02328844], dtype=float32),
 'france': array([ 0.01525989,  0.0218841 ,  0.00681947, ..., -0.04391771,
        -0.02820129,  0.02574923], dtype=float32),
 'germany': array([ 0.01111386,  0.01362935,  0.00545584, ..., -0.03046325,
        -0.0210385 ,  0.01710157], dtype=float32),
 'india': array([ 0.01481213,  0.01650877,  0.008064  , ..., -0.03975716,
        -0.02910383,  0.02112232], dty

In [165]:
def sim(b):
    sims = []
    for each in g20:
        a = g20_mean_emb[each]
        sims.append(np.dot(a, b)/(norm(a)*norm(b)))
    return sims

In [166]:
sim(data['mean_emb'][0])

[0.9966552,
 0.9954498,
 0.99931955,
 0.99488163,
 0.997215,
 0.9896934,
 0.9968328,
 0.99961454,
 0.99985653,
 0.9949286,
 0.98023343,
 0.9989037,
 0.9888792,
 0.99979323,
 0.99491817,
 0.9983936,
 0.9997712,
 0.94626665,
 0.997785,
 0.99946123]

In [167]:
data['sims'] = data['mean_emb'].apply(lambda x: sim(x))

In [168]:
sims = pd.DataFrame(data.sims.tolist(), index= data.index)
sims['id'] = data.index
sims

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,id
0,0.996655,0.995450,0.999320,0.994882,0.997215,0.989693,0.996833,0.999615,0.999857,0.994929,...,0.998904,0.988879,0.999793,0.994918,0.998394,0.999771,0.946267,0.997785,0.999461,0
1,0.999876,0.999552,0.999346,0.999294,0.999811,0.997180,0.999719,0.998932,0.998430,0.999366,...,0.999517,0.996825,0.998066,0.999473,0.999537,0.997079,0.966031,0.999879,0.996359,1
2,0.999472,0.999810,0.997153,0.999584,0.999214,0.999321,0.999151,0.996381,0.995503,0.999602,...,0.997772,0.999068,0.994857,0.999931,0.998188,0.993326,0.975108,0.998957,0.992333,2
3,0.995488,0.996607,0.990179,0.996918,0.994651,0.999418,0.995030,0.988784,0.987069,0.996854,...,0.991356,0.999471,0.986360,0.997141,0.992113,0.983952,0.987683,0.993855,0.982745,3
4,0.996738,0.997755,0.992083,0.997795,0.996090,0.999788,0.996235,0.990819,0.989362,0.997730,...,0.993198,0.999824,0.988545,0.998141,0.993964,0.986293,0.984982,0.995419,0.985090,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,0.994372,0.992669,0.998142,0.992280,0.994954,0.985817,0.994740,0.998663,0.999042,0.992406,...,0.997348,0.984863,0.999308,0.992111,0.996501,0.999684,0.938578,0.995725,0.999579,2220
2221,0.999116,0.999584,0.996232,0.999470,0.998753,0.999723,0.998767,0.995341,0.994301,0.999443,...,0.996974,0.999574,0.993658,0.999751,0.997428,0.991941,0.977875,0.998388,0.990944,2221
2222,0.999945,0.999666,0.998981,0.999655,0.999729,0.997828,0.999852,0.998484,0.997775,0.999738,...,0.999162,0.997443,0.997535,0.999718,0.999149,0.996467,0.968835,0.999711,0.995754,2222
2223,0.998841,0.997833,0.999895,0.997936,0.998874,0.993951,0.999079,0.999843,0.999459,0.998028,...,0.999579,0.993261,0.999663,0.997687,0.999047,0.999254,0.957757,0.999168,0.998917,2223


In [169]:
def lookup(n, col):
    relevant = sims.loc[:,[col,'id']].sort_values(by = col).iloc[-n]
    id = int(relevant['id'])
    print(data.loc[id, 'title'] + "\n")
    print(str(relevant[col]) + "\n")
    print(data.loc[id, 'contents'] + "\n")

In [170]:
"Argentina, Australia, Brazil, Canada, China, France, Germany, India, Indonesia, Italy, Japan, Republic of Korea, Mexico, Russia, Saudi Arabia, South Africa, Turkey, United Kingdom, United States, European Union"

'Argentina, Australia, Brazil, Canada, China, France, Germany, India, Indonesia, Italy, Japan, Republic of Korea, Mexico, Russia, Saudi Arabia, South Africa, Turkey, United Kingdom, United States, European Union'

In [171]:
lookup(50, 7)

DVD copy protection strengthened

0.9999194741249084

DVDs will be harder to copy thanks to new anti-piracy measures devised by copy protection firm Macrovision.

The pirated DVD market is enormous because current copy protection was hacked more than five years ago. Macrovision says its new RipGuard technology will thwart most, but not all, of the current DVD ripping (copying) programs used to pirate DVDs. "RipGuard is designed to... reduce DVD ripping and the resulting supply of illegal peer to peer," said the firm. Macrovision said the new technology will work in "nearly all" current DVD players when applied to the discs, but it did not specify how many machines could have a problem with RipGuard. The new technology will be welcomed by Hollywood film studios which are increasingly relying on revenue from DVD sales.

The film industry has stepped up efforts to fight DVD piracy in the last 12 months, taking legal action against websites which offer pirated copies of DVD movies for down

In [172]:
sum((sims[1]>0.9995).astype(int))

472

In [173]:
sum(data['australia'])

85

In [174]:
final = (sims>0.9995).astype(int).iloc[:,:20]
final.columns = g20
final

,argentina,australia,brazil,canada,china,france,germany,india,indonesia,italy,japan,korea,mexico,russia,arabia,africa,turkey,uk,america,eu
0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0
1,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0
2,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
2221,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2222,1,1,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0
2223,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0


In [177]:
data = data.loc[:,'argentina':'european union']
data.columns = g20
final = (data.add(final)>=1).astype(int)

KeyError: 'european union'

In [180]:
final['theme'] = data['theme']

In [181]:
final.to_pickle("./topics.pkl")
final.to_csv("./topics.csv")